In [2]:
import torch
a=torch.tensor(11)
a.shape

torch.Size([])

In [7]:
weight=torch.ones(16,768,16,16)
weight*torch.tensor(12)

tensor([[[[12., 12., 12.,  ..., 12., 12., 12.],
          [12., 12., 12.,  ..., 12., 12., 12.],
          [12., 12., 12.,  ..., 12., 12., 12.],
          ...,
          [12., 12., 12.,  ..., 12., 12., 12.],
          [12., 12., 12.,  ..., 12., 12., 12.],
          [12., 12., 12.,  ..., 12., 12., 12.]],

         [[12., 12., 12.,  ..., 12., 12., 12.],
          [12., 12., 12.,  ..., 12., 12., 12.],
          [12., 12., 12.,  ..., 12., 12., 12.],
          ...,
          [12., 12., 12.,  ..., 12., 12., 12.],
          [12., 12., 12.,  ..., 12., 12., 12.],
          [12., 12., 12.,  ..., 12., 12., 12.]],

         [[12., 12., 12.,  ..., 12., 12., 12.],
          [12., 12., 12.,  ..., 12., 12., 12.],
          [12., 12., 12.,  ..., 12., 12., 12.],
          ...,
          [12., 12., 12.,  ..., 12., 12., 12.],
          [12., 12., 12.,  ..., 12., 12., 12.],
          [12., 12., 12.,  ..., 12., 12., 12.]],

         ...,

         [[12., 12., 12.,  ..., 12., 12., 12.],
          [12., 12., 1

In [9]:
import torch.nn as nn
picture_in=torch.rand(1,3,224,224)

nn.conv(picture_in)

AttributeError: module 'torch.nn' has no attribute 'conv'

In [12]:
6.4712/0.0509

127.13555992141453